# TOP

In [1]:
import functools
import itertools
import pickle

import numpy as np
import pathlib
import scipy.integrate as integrate
import scipy.optimize as optimize
import scipy.stats as stats
from tqdm.notebook import tqdm

from flexfrac1d.model.model import Ice, Floe, Ocean, FloatingIce, WavesUnderFloe, WavesUnderIce
from flexfrac1d.model.frac_handlers import BinaryFracture, BinaryStrainFracture

# Tank experiment, sine waves

In [2]:
with open("res_baptiste/params_tot_AUVITY.pkl", "rb") as f:
    pickled_dict = pickle.load(f)

print(pickled_dict.keys())

experimental_wavelengths = pickled_dict["lambda_s"]
experimental_thresholds = pickled_dict["a_s"]
experimental_wavenumbers = 2 * np.pi / experimental_wavelengths
experimental_flex_lengths = pickled_dict["L_d"]

_temp_wl, _temp_kappa_c = np.loadtxt("res_baptiste/kappa_c.txt", usecols=range(1, 11))
assert np.allclose(experimental_wavelengths - _temp_wl, 0)
experimental_curvature_thresholds = _temp_kappa_c

dict_keys(['L_d', 'D', 'l_s', 'a_s', 'k_s', 'h', 'lambda_s', 'l_crack', 'dates', 'path', 'nom_exps', 'rho'])


In [3]:
def threshold_search(
    wui,
    length,
    amplitude_a=1e-4,
    amplitude_b=1e-1,
    atol=1e-6,  # 1 µm
    rtol=1e-3,
    phase=0,
    fracture_handler=None,
    linear_curvature=True,
):
    if fracture_handler is None:
        fracture_handler = BinaryFracture()
    threshold = np.nan  
    n = 0
    # (b - a) / a > rtol
    while amplitude_b - amplitude_a > atol or amplitude_b / amplitude_a > 1 + rtol:
        n += 1
        amplitude = np.sqrt(amplitude_a * amplitude_b)  # geometric mean to find lower values quicker?
        xf, _ = compute_split(amplitude, phase, wui, length, fracture_handler, linear_curvature)
        
        if xf is None:
            # No fracture, increase the amplitude
            amplitude_a = amplitude
        else:
            # Fracture, lower the amplitude, and update the threshold
            if threshold is np.nan:
                threshold = amplitude
            else:
                if amplitude < threshold:  # Redondant, on cherche a priori dans un intervalle majoré par threshold
                    threshold = amplitude
            amplitude_b = amplitude
    return threshold, n


def compute_split(amplitude, phase, wui, length, fracture_handler, linear_curvature=True):
    complex_amplitude = np.atleast_1d(amplitude * np.exp(1j * phase))
    wuf = WavesUnderFloe(
        left_edge=0,
        length=length,
        wui=wui,
        edge_amplitudes=complex_amplitude,
    )
    xf = fracture_handler.search(
        wuf=wuf,
        growth_params=None,
        an_sol=None,
        num_params=None,
        linear_curvature=linear_curvature,
    )
    return xf, wuf


def compute_dissipation_length(wuf, xf, fh):
    left_fragment, right_fragment = fh.split(wuf, xf)
    length_left = (
        integrate.quad(
            lambda x: x * (wuf.curvature(xf - x, None, True, None) - left_fragment.curvature(xf - x, None, True, None))**2, 0, xf
        )[0] / integrate.quad(
            lambda x: (wuf.curvature(xf - x, None, True, None) - left_fragment.curvature(xf - x, None, True, None))**2, 0, xf
        )[0]
    )
    length_right = (
        integrate.quad(
            lambda x: x * (wuf.curvature(x + xf, None, True, None) - right_fragment.curvature(x, None, True, None))**2, 0, length - xf
        )[0] / integrate.quad(
            lambda x: (wuf.curvature(x + xf, None, True, None) - right_fragment.curvature(x, None, True, None))**2, 0, length - xf
        )[0]
    )
    return length_left + length_right

In [4]:
material_constant = 2.8e-4

density = 680
poissons_ratio = .4
thickness = 158e-6
flexural_rigidity = 3.1e-5
youngs_modulus = 12 * (1 - poissons_ratio**2) * flexural_rigidity / thickness**3
print(f"Y = {youngs_modulus:e}")

energy_release_rate = flexural_rigidity * material_constant / (2 * thickness**2)
print(f"G = {energy_release_rate}")
fracture_thoughness = (energy_release_rate * youngs_modulus / (1 - poissons_ratio**2))**.5
print(f"K = {fracture_thoughness}")

depth = 11e-2
fluid_density = 1e3

gravity = 9.8

varnish = Ice(
    density=density,
    frac_toughness=fracture_thoughness,
    poissons_ratio=poissons_ratio,
    thickness=thickness,
    youngs_modulus=youngs_modulus,
)
assert np.isclose(varnish.frac_energy_rate - energy_release_rate, 0)
print(f"D = {varnish.flex_rigidity}")
tank = Ocean(
    depth=depth,
    density=fluid_density,
)
floating_varnish = FloatingIce.from_ice_ocean(ice=varnish, ocean=tank, gravity=gravity)
flexural_length = floating_varnish.elastic_length
print(f"L_D = {flexural_length}")

Y = 7.922294e+07
G = 0.17385034449607434
K = 4049.240922863745
D = 3.1e-05
L_D = 0.0074995275441722764


In [12]:
import json

tank_params = {"depth": depth, "density": fluid_density}
varnish_params = {
    "density": density,
    "energy_release_rate": energy_release_rate,
    "flexural_length": flexural_length,
    "flexural_rigidity": flexural_rigidity,
    "poissons_ratio": poissons_ratio,
    "thickness": thickness,
    "frac_toughness": fracture_thoughness,
    "youngs_modulus": youngs_modulus,
}
reference_experiment_parameters = {"gravity": gravity, "tank": tank_params, "varnish": varnish_params}
with open("reference_experiment_parameters.json", "w") as f:
    json.dump(reference_experiment_parameters, f)

## Simple comparison to data, single mode

In [5]:
path = "results/stationnary_simple_comparison.npz"
rerun = True

if rerun:
    fracture_handler = BinaryFracture()
    max_wave_slope = .5
    harmonic_number = 3
    phase = 0
    wavenumbers = np.geomspace(
        experimental_wavenumbers.min() / 1.5,
        experimental_wavenumbers.max() * 4,
        256,
    )

    lengths = harmonic_number * np.pi / wavenumbers
    amplitude_thresholds = np.full(wavenumbers.size, np.nan)
    curvature_thresholds = np.full(wavenumbers.size, np.nan)
    curvatures_at_half_point = np.full(wavenumbers.size, np.nan)
    normalised_max_curvature_coords = np.full(wavenumbers.size, np.nan)
    max_curvatures = np.full(wavenumbers.size, np.nan)
    relaxation_lengths = np.full(wavenumbers.size, np.nan)
    normalised_fractures = np.full(wavenumbers.size, np.nan)  # coordinate
    wuis = [
        WavesUnderIce(floating_varnish, np.atleast_1d(k), 0)
        for k in wavenumbers
    ]
    
    for j, (wui, length) in enumerate(tqdm(zip(wuis, lengths), total=len(wuis))):
        amplitude_threshold, n = threshold_search(
            wui,
            length,
            atol=1e-7,  # 0.1 µm
            rtol=1e-4,
            amplitude_b=max_wave_slope / wui.wavenumbers[0],
            fracture_handler=fracture_handler,
        )
        amplitude_thresholds[j] = amplitude_threshold
        if amplitude_threshold is not np.nan:
            xf, wuf = compute_split(amplitude_threshold, phase, wui, length, fracture_handler)
            curvature_thresholds[j] = wuf.curvature(xf, None, True, None)
            normalised_fractures[j] = xf / length

            curvatures_at_half_point[j] = wuf.curvature(length / 2, None, True, None)
            # Taking advantage of the fact kappa(L / 2) is a maximum,
            # and we look for a negative minimum
            opt = optimize.minimize_scalar(
                lambda x: wuf.curvature(x, None, True, None),
                bounds=(0, length / 2),
            )
            normalised_max_curvature_coords[j] = opt.x / length
            max_curvatures[j] = wuf.curvature(opt.x, None, True, None)
            relaxation_lengths[j] = compute_dissipation_length(wuf, xf, fracture_handler)

    np.savez(
        path,
        wavenumbers=wavenumbers,
        amplitude_thresholds=amplitude_thresholds,
        curvature_thresholds=curvature_thresholds,
        normalised_fractures=normalised_fractures,
        curvatures_at_half_point=curvatures_at_half_point,
        normalised_max_curvature_coords=normalised_max_curvature_coords,
        max_curvatures=max_curvatures,
        relaxation_lengths=relaxation_lengths,
    )
else:
    loaded = np.load(path)
    (
        wavenumbers,
        amplitude_thresholds,
        curvature_thresholds,
        normalised_fractures,
        curvatures_at_half_point,
        normalised_max_curvature_coords,
        max_curvatures,
        relaxation_lengths,
    ) = (
        loaded[k] for k in (
            "wavenumbers",
            "amplitude_thresholds",
            "curvature_thresholds",
            "normalised_fractures",
            "curvatures_at_half_point",
            "normalised_max_curvature_coords",
            "max_curvatures",
            "relaxation_lengths",
        )
    )
nondim = wavenumbers * flexural_length

  0%|          | 0/256 [00:00<?, ?it/s]

## Ensemble comparison to data

In [612]:
_h = pickled_dict["h"]
_D = pickled_dict["D"]
_Y = _D / _h**3 * (12 * (1 - .4**2))
print(f"h: {_h.mean():.2e} pm {_h.std():.2e}")
print(f"Y: {_Y.mean():.2e} pm {_Y.std():.2e}")

h: 1.04e-04 pm 4.95e-05
Y: 6.87e+07 pm 6.33e+07


In [6]:
def gen_sample(n_samples):
    seed = 0x3838
    rng = np.random.default_rng(seed=seed)
    lhc = stats.qmc.LatinHypercube(2, seed=rng)
    samples = lhc.random(n=n_samples)
    
    youngs_modulus_norm_params = 70e6, 14e6
    thickness_norm_params = 100e-6, 20e-6
    
    thicknesses = stats.norm.ppf(samples[:, 0], *thickness_norm_params)
    youngs_moduli = stats.norm.ppf(samples[:, 1], *youngs_modulus_norm_params)
    assert not np.any((thicknesses < 0) | (youngs_moduli < 0))
    
    return thicknesses, youngs_moduli
    

def gen_toughnesses(thicknesses, youngs_moduli, poissons_ratio, material_constant):
    flexural_rigidities = youngs_moduli * thicknesses**3 / (12 * (1 - poissons_ratio**2))
    energy_release_rates = material_constant * flexural_rigidities / (2 * thicknesses**2)
    
    toughnesses = (energy_release_rates * youngs_moduli / (1 - poissons_ratio**2))**.5
    return toughnesses


def gen_varnishes(
    thicknesses,
    youngs_moduli,
    frac_toughnesses,
    density,
    poissons_ratio,
    tank,
    gravity,    
):
    floating_varnishes = [
        FloatingIce.from_ice_ocean(
            ice=Ice(
                density=density,
                frac_toughness=_K,
                poissons_ratio=poissons_ratio,
                thickness=_h,
                youngs_modulus=_E,
            ),
            ocean=tank,
            gravity=gravity
        ) for _h, _E, _K in zip(thicknesses, youngs_moduli, frac_toughnesses)
    ]
    return floating_varnishes

In [ ]:
harmonic_number = 3  # to be varied
path = f"results/stationnary_ensemble_comparison_nondim_hn{harmonic_number:02d}.npz"
rerun = True

if rerun:
    n_samples = 128
    thicknesses, youngs_moduli = gen_sample(n_samples)
    toughnesses = gen_toughnesses(thicknesses, youngs_moduli, poissons_ratio, material_constant)
    floating_varnishes = gen_varnishes(thicknesses, youngs_moduli, toughnesses, density, poissons_ratio, tank, gravity)
    assert np.allclose(
        np.array([
            _fv.frac_energy_rate / (_fv.flex_rigidity / (2 * _fv.thickness**2))
            for _fv in floating_varnishes
        ]) - material_constant, 0
    )
    flex_lengths, flexural_rigidities, energy_release_rates = map(
        np.array,
        zip(*[
            (_fv.elastic_length, _fv.flex_rigidity, _fv.frac_energy_rate)
            for _fv in floating_varnishes
        ]),
    )
    attenuation = 0
    phase = 0
    
    fracture_handler = BinaryFracture()
    max_wave_slope = .5
    nondim = np.geomspace(8e-2, 2.5, 256)
    wavenumbers = nondim / flex_lengths[:, None]
    lengths = harmonic_number * np.pi / wavenumbers
    amplitude_thresholds = np.full((n_samples, nondim.size), np.nan)
    curvature_thresholds = np.full((n_samples, nondim.size), np.nan)
    energy_dissipation_lengths = np.full((n_samples, nondim.size), np.nan)
    
    for i, _fv in enumerate(tqdm(floating_varnishes)):
        for j, (_wn, length) in enumerate(
            tqdm(zip(wavenumbers[i], lengths[i]), total=nondim.size, leave=False)
        ):
            wui = WavesUnderIce(_fv, np.atleast_1d(_wn), 0)
            amplitude_threshold, n = threshold_search(
                wui,
                length,
                amplitude_b=max_wave_slope / wui.wavenumbers[0],
                fracture_handler=fracture_handler,
            )
            amplitude_thresholds[i, j] = amplitude_threshold
            if amplitude_threshold is not np.nan:
                xf, wuf = compute_split(amplitude_threshold, phase, wui, length, fracture_handler)
                curvature_thresholds[i, j] = wuf.curvature(xf, None, True, None)
                energy_dissipation_lengths[i, j] = compute_dissipation_length(wuf, xf, fracture_handler)
    np.savez(
        path,
        nondim=nondim,
        thicknesses=thicknesses,
        youngs_moduli=youngs_moduli,
        flexural_rigidities=flexural_rigidities,
        flex_lengths=flex_lengths,
        lengths=lengths,
        energy_release_rates=energy_release_rates,
        wavenumbers=wavenumbers,
        amplitude_thresholds=amplitude_thresholds,
        curvature_thresholds=curvature_thresholds,
        energy_dissipation_lengths=energy_dissipation_lengths,
    )
else:
    loaded = np.load(path)
    (
        nondim,
        thicknesses,
        youngs_moduli,
        flexural_rigidities,
        flex_lengths,
        lengths,
        energy_release_rates,
        wavenumbers,
        amplitude_thresholds,
        curvature_thresholds,
        energy_dissipation_lengths,
    ) = (
        loaded[k] for k in (
            "nondim",
            "thicknesses",
            "youngs_moduli",
            "flexural_rigidities",
            "flex_lengths",
            "lengths",
            "energy_release_rates",
            "wavenumbers",
            "amplitude_thresholds",
            "curvature_thresholds",
            "energy_dissipation_lengths",
        )
    )

In [28]:
pickled_dict["lambda_s"]

array([0.4312    , 0.29268525, 0.10473971, 0.23693369, 0.14303956,
       0.54114617, 0.52461165, 0.36301367, 0.20899214, 0.06195224])

In [29]:
2 * np.pi / pickled_dict["lambda_s"]

array([ 14.5713945 ,  21.46737962,  59.98856945,  26.51874984,
        43.92620751,  11.61088375,  11.97683154,  17.30839872,
        30.06421838, 101.41982448])

In [58]:
wavenumbers = 2 * np.pi / pickled_dict["lambda_s"]
sort_key = np.argsort(wavenumbers)
harmonic_numbers = np.ones_like(wavenumbers)
harmonic_numbers[sort_key[-2:]] = 2
harmonic_numbers[sort_key[-4:-2]] = 3
(harmonic_numbers * np.pi / wavenumbers).min()

0.06195224

In [62]:
2 * np.pi / wavenumbers.max()

0.06195224

In [60]:
pickled_dict["a_s"].min()

0.00162004